In [1]:
from clipboard import *

In [19]:
s = 'Midnaught (Blood) (BIS list - highlighted if have)\r\nTyriel (Unholy)\r\nTyriel (Frost)\r\nLilshadow (Holy)\r\nHawaiinroll (Retribution)\r\nRefanone (Feral)\r\nHrothgarth (Feral)\r\nVoomkin (Balance)\r\nWs (Restoration)\r\nLilboov (Guardian)\r\nSonohana\r\nNvrwastaken\r\nJorelboov (Beast Mastery)\r\nSleepypan\r\nKandlicious\r\nDott (Windwalker)\r\nGoblinguide (Fire)\r\nSwagbag (Frost)\r\nNamyio (Frost)\r\nDookú (Elemental)\r\nGrasping (Restoration)\r\nStormblast (Enhance)\r\nKôrboov (Affliction) - Haste=Mastery>Crit>Vers\r\nRenewed (Discipline)\r\nSìeara (Holy - Int > Mastery 40% > Crit = Has > Vers)\r\nSìeara (Discipline - Int > Haste > Mastery = Crit = Vers)\r\nNghtly wait how do iplay this class\r\nPhue (Windwalker)'

In [20]:
for c in s.splitlines():
    print(c.split()[0])

Midnaught
Tyriel
Tyriel
Lilshadow
Hawaiinroll
Refanone
Hrothgarth
Voomkin
Ws
Lilboov
Sonohana
Nvrwastaken
Jorelboov
Sleepypan
Kandlicious
Dott
Goblinguide
Swagbag
Namyio
Dookú
Grasping
Stormblast
Kôrboov
Renewed
Sìeara
Sìeara
Nghtly
Phue


In [54]:
class _ElementContainer():
    def __init__(self):
        self.items = []
        
    def _add_item(self, kls, *a, **k):
        if len(a) == 1 and isinstance(a[0], kls):
            i = a[0]
        else:
            i = kls(*a, **k)
        return self.add_item(i)
    
    def add_item(self, i):
        self.items.append(i)
        return i
    
    def item_adder(self, kls):
        def func(*a, **k):
            return self._add_item(kls, *a, **k)
        return func
    


class HTMLDoc(_ElementContainer):
    def __init__(self):
        super().__init__()
        self.table = self.item_adder(Table)
        
    def _dump(self, b, i=0):
        s = "  "
        b.append("<!DOCTYPE html>\n")
        b.append("%s<html>\n"%(s*i))
        b.append("%s<body>\n"%(s*(i+1)))
        for item in self.items:
            item._dump(b, i+1)
        b.append("%s</body>\n"%(s*(i+1)))
        b.append("%s</html>\n"%(s*i))
        return b
                
    def dump(self):
        b = []
        self._dump(b)
        return "".join(b)
    

def _nl(b):
    b.append("\n")
    
        
class Element(_ElementContainer):
    name = "unknown"
    def __init__(self, **attrib):
        super().__init__()
        self.attrib = attrib
            
    def _open(self, b, indent=0, nl=True):
        a = " ".join("%s=\"%s\""%(k,v) for k,v in self.attrib.items())
        b.append("%s<%s%s%s>%s" % ("  "*indent,self.tag, " " if a else "", a, "\n" if nl else ""))
        return b
    
    def _close(self, b, indent=0, nl=True):
        b.append("%s</%s>%s"%("  "*indent, self.tag, "\n" if nl else ""))
        return b
    
    def _dump(self, b, indent=0):
        self._open(b, indent)
        for r in self.items:
            r._dump(b, indent+1)
        self._close(b, indent)
        return b
        
    def dump(self):
        return "".join(self._dump([]))

        
        
# class HTML(Element):
#     tag = "html"
#     def __init__(self, **a):
#         super().__init__(**a)
#         self.body = self.item_adder(Body)
        
        
# class Body(Element):
#     tag = "body"
        
        
class Table(Element):
    tag = "table"
    def __init__(self, **a):
        super().__init__(**a)
        self.row = self.tr = self.item_adder(Row)
        self.style = self.item_adder(Style)
        
        
class Row(Element):
    tag = "tr"
    def __init__(self, **attrib):
        super().__init__(**attrib)
        self.th = self.header = self.item_adder(TableHeader)
        self.td = self.data = self.column = self.item_adder(TableData)
    
    
class TableData(Element):
    tag = "td"
    def __init__(self, text, **a):
        super().__init__(**a)
        self.text = text
        
    def get_text(self):
        return self.text
    
    def _dump(self, b, indent=0, samerow=True):
        self._open(b, indent, not samerow)
        b.append(self.get_text())
        self._close(b, 0 if samerow else indent, True)
        
class TableHeader(TableData):
    tag = "th"
    
class Style(Element):
    tag = "style"
    def __init__(self):
        super().__init__()
        self.selector = self.item_adder(Selector)
        
    def _dump(self, b, indent=0):
        self._open(b, indent)
        for s in self.items:
            b.append("%s%s {\n" %("  "*(indent+1), s.name))
            for k,v in s.props.items():
                b.append("%s%s: %s;\n"%("  "*(indent+2), k,v))
            b.append("%s}"%("  "*(indent+1)))
        self._close(b, indent)
        
    
        
class Selector():
    """ I don't understand CSS yet but this class will 
    handle some *very* basic style properties until I 
    learn more. 
    """
    def __init__(self, s, p=None, **props):
        if p is not None:
            p.update(props)
        self.name = s
        self.props = p
        
    def prop(self, k, v):
        self.props[k] = v
        
    property = prop
        

# Example CSS style below for reference
"""<style>
table {
    font-family: arial, sans-serif;
    border-collapse: collapse;
    width: 100%;
}

td, th {
    border: 1px solid #dddddd;
    text-align: left;
    padding: 8px;
}

tr:nth-child(even) {
    background-color: #dddddd;
}
</style>"""
    

In [ ]:
gear_slots = [
    "helm",
    "neck",
    "shoulder",
    "cloak",
    "chest",
    "bracer",
    "gloves",
    "waist",
    "legs",
    "boots",
    "ring1",
    "ring2",
    "trinket1",
    "trinket2",
    "relic1",
    "relic2",
    "relic3"
]

In [55]:
doc = HTMLDoc()
t=doc.table()
style = t.style()
style.selector("table", {
    "font-family": "arial, sans-serif",
    "border-collapse": "collapse",
})
style.selector("td, th",{
    "border": "1px solid #dddddd",
    "text-align": "left",
})
for i in t.add_item(Player("Name", "Class", "Rank")).items:
    i.tag = "th"

for name in s.splitlines():
    n = name.split()[0]
    p = Player(n, "probably druid", "gay")
    t.add_item(p)

s2 = doc.dump()

with open("test.html", 'w') as f:
    f.write(s2)

import webbrowser
webbrowser.open("test.html")

True

In [56]:
doc = HTMLDoc()
t=doc.table()
style = t.style()
style.selector("table", {
    "font-family": "arial, sans-serif",
    "border-collapse": "collapse",
})
style.selector("td, th",{
    "border": "1px solid #dddddd",
    "text-align": "left",
})